# Veteran Suicides Analysis

I downloaded a dataset from the Department of Veterans Affairs with veterans suicide statistics from 2001-2021. I wanted to answer a few questions from this dataset:

1. Is there a significant change in the number of suicides over time?
2. Is there a significant difference in suicide rates between veterans and non-veterans?
3. What is the breakdown in suicide rates by age group?
4. Is there a significant difference in suicide rates between male veterans and female veterans?
5. What is the breakdwon in suicide rates by demographic group?

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import statsmodels.api as sm

veterans = pd.read_excel('va_suicides_prepped.xlsx', sheet_name='Veteran')
non_veterans = pd.read_excel('va_suicides_prepped.xlsx', sheet_name='Non-Veteran')
by_demos = pd.read_excel('va_suicides_prepped.xlsx', sheet_name='Veteran Race & Ethnicity')

In [4]:
new_cols = ['year', 'age_group', 'deaths', 'population', 'rate_per_100k', 'male_deaths', 'male_population', 'male_rate_per_100k', 'female_age_group', 'female_deaths', 'female_population', 'female_rate_per_100k']
veterans.columns = new_cols
veterans['female_deaths'] = pd.to_numeric(veterans['female_deaths'], errors='coerce')
veterans['female_population'] = pd.to_numeric(veterans['female_population'], errors='coerce') 
veterans['female_rate_per_100k'] = pd.to_numeric(veterans['female_rate_per_100k'], errors='coerce')
veterans.info()
veterans.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  105 non-null    int64  
 1   age_group             105 non-null    object 
 2   deaths                105 non-null    int64  
 3   population            105 non-null    int64  
 4   rate_per_100k         105 non-null    float64
 5   male_deaths           105 non-null    int64  
 6   male_population       105 non-null    int64  
 7   male_rate_per_100k    105 non-null    float64
 8   female_age_group      105 non-null    object 
 9   female_deaths         84 non-null     float64
 10  female_population     84 non-null     float64
 11  female_rate_per_100k  84 non-null     float64
dtypes: float64(5), int64(5), object(2)
memory usage: 10.0+ KB


,year,age_group,deaths,population,rate_per_100k,male_deaths,male_population,male_rate_per_100k,female_age_group,female_deaths,female_population,female_rate_per_100k
0,2001,18-34,616,2623000,23.484560,586,2227000,26.313426,18-34,30.0,396000.0,7.575758
1,2001,35-54,2510,8957000,28.022775,2403,8199000,29.308452,35-54,107.0,758000.0,14.116095
2,2001,55-74,1693,9761000,17.344534,1682,9512000,17.682927,55+,17.0,466000.0,3.648069
3,2001,75+,1178,4457000,26.430334,1172,4240000,27.641509,.,NaN,NaN,NaN
4,2001,All,6000,25798000,23.257617,5846,24178000,24.179006,All,154.0,1620000.0,9.506173


In [5]:
all_veterans = veterans[veterans['age_group'] == 'All']
all_veterans['deaths_growth'] = all_veterans['deaths'].pct_change()
all_veterans['rate_growth'] = all_veterans['rate_per_100k'].pct_change()
all_veterans['male_rate_growth'] = all_veterans['male_rate_per_100k'].pct_change()
all_veterans['female_rate_growth'] = all_veterans['female_rate_per_100k'].pct_change()
all_veterans

,year,age_group,deaths,population,rate_per_100k,male_deaths,male_population,male_rate_per_100k,female_age_group,female_deaths,female_population,female_rate_per_100k,deaths_growth,rate_growth,male_rate_growth,female_rate_growth
4,2001,All,6000,25798000,23.257617,5846,24178000,24.179006,All,154.0,1620000.0,9.506173,NaN,NaN,NaN,NaN
9,2002,All,6142,25423000,24.159226,6008,23752000,25.294712,All,134.0,1671000.0,8.019150,0.023667,0.038766,0.046144,-0.156427
14,2003,All,6008,25039000,23.994568,5857,23331000,25.103939,All,151.0,1708000.0,8.840749,-0.021817,-0.006816,-0.007542,0.102455
19,2004,All,6004,24800000,24.209677,5837,23046000,25.327606,All,167.0,1754000.0,9.521095,-0.000666,0.008965,0.008910,0.076956
24,2005,All,6126,24616000,24.886253,5937,22793000,26.047471,All,189.0,1823000.0,10.367526,0.020320,0.027946,0.028422,0.088901
29,2006,All,6035,24163000,24.976203,5862,22346000,26.232883,All,173.0,1817000.0,9.521189,-0.014855,0.003614,0.007118,-0.081633
34,2007,All,6249,23676000,26.393817,6063,21906000,27.677349,All,186.0,1770000.0,10.508475,0.035460,0.056759,0.055063,0.103694
39,2008,All,6567,23364000,28.107345,6359,21624000,29.407140,All,208.0,1740000.0,11.954023,0.050888,0.064922,0.062498,0.137560
44,2009,All,6519,23026000,28.311474,6293,21246000,29.619693,All,226.0,1780000.0,12.696629,-0.007309,0.007262,0.007228,0.062122
49,2010,All,6545,22767000,28.747749,6313,20981000,30.089128,All,232.0,1786000.0,12.989922,0.003988,0.015410,0.015849,0.023100


## Question 1

Studying veteran suicides and suicide rates over the time period.

In [6]:
min_deaths = all_veterans['deaths'].min()
min_death_year = all_veterans.loc[all_veterans['deaths'] == min_deaths, 'year'].values[0]
max_deaths = all_veterans['deaths'].max()
max_death_year = all_veterans.loc[all_veterans['deaths'] == max_deaths, 'year'].values[0]
print(f'Min deaths: {min_deaths} in the year {min_death_year}')
print(f'Max deaths: {max_deaths} in the year {max_death_year}')

Min deaths: 6000 in the year 2001
Max deaths: 6718 in the year 2018


In [7]:
fig = px.line(data_frame=all_veterans, x='year', y='deaths', title='Veteran Suicides 2001-2021', range_y=(0, 7000))
fig.show()

In [8]:
fig = px.line(data_frame=all_veterans, x='year', y='rate_per_100k', title='Veteran Suicide Rates 2001-2021', range_y=(0, 40))
fig.show()

In [9]:
min_rate = all_veterans['rate_per_100k'].min()
min_rate_year = all_veterans.loc[all_veterans['rate_per_100k'] == min_rate, 'year'].values[0]
max_rate = all_veterans['rate_per_100k'].max()
max_rate_year = all_veterans.loc[all_veterans['rate_per_100k'] == max_rate, 'year'].values[0]
print(f'Min death rate: {min_rate:.2f} in the year {min_rate_year}')
print(f'Max death rate: {max_rate:.2f} in the year {max_rate_year}')

Min death rate: 23.26 in the year 2001
Max death rate: 33.93 in the year 2021


In [10]:
avg_deaths_growth = all_veterans['deaths_growth'].mean() * 100
avg_rate_growth = all_veterans['rate_growth'].mean() * 100
print(f"Average growth rate in suicides: {avg_deaths_growth:.2f}%")
print(f"Average growth rate in rates per 100k population: {avg_rate_growth:.2f}%.")

Average growth rate in suicides: 0.34%
Average growth rate in rates per 100k population: 1.93%.


In [11]:
x = all_veterans['year']
y = all_veterans['rate_per_100k']
X = sm.add_constant(x)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          rate_per_100k   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     658.6
Date:                Sat, 25 May 2024   Prob (F-statistic):           3.29e-16
Time:                        18:26:05   Log-Likelihood:                -17.729
No. Observations:                  21   AIC:                             39.46
Df Residuals:                      19   BIC:                             41.55
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1071.8749     42.886    -24.994      0.000   -1161.636    -982.114
year           0.5473      0.021     25.663      0.000       0.503       0.592
==============================================================================
Omnibus:                        0.487   Durbin-Watson:                   1.211
Prob(Omnibus):                  0.784   Jarque-Bera (JB):                0.565
Skew:                           0.046   Prob(JB):                        0.754
Kurtosis:                       2.202   Cond. No.                     6.68e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.68e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
next_five = pd.DataFrame({'year': [2022, 2023, 2024, 2025, 2026]})
next_five = sm.add_constant(next_five)
predictions = model.predict(next_five)
forecast = pd.DataFrame({'year': next_five['year'], 'forecast': predictions})
forecast['forecast'] = forecast['forecast'].round(1)
forecast

,year,forecast
0,2022,34.7
1,2023,35.2
2,2024,35.8
3,2025,36.3
4,2026,36.9


## Question 2

Determining whether there is a significant difference in suicide rates between veterans and non-veterans.

In [10]:
non_veterans.columns = new_cols
non_veterans.info()
non_veterans.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  105 non-null    int64  
 1   age_group             105 non-null    object 
 2   deaths                105 non-null    int64  
 3   population            105 non-null    int64  
 4   rate_per_100k         105 non-null    float64
 5   male_deaths           105 non-null    int64  
 6   male_population       105 non-null    int64  
 7   male_rate_per_100k    105 non-null    float64
 8   female_age_group      105 non-null    object 
 9   female_deaths         105 non-null    object 
 10  female_population     105 non-null    object 
 11  female_rate_per_100k  105 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 10.0+ KB


,year,age_group,deaths,population,rate_per_100k,male_deaths,male_population,male_rate_per_100k,female_age_group,female_deaths,female_population,female_rate_per_100k
0,2001,18-34,7677,64841174,11.839699,6428,32011909,20.080027,18-34,1249,32829265,3.804532
1,2001,35-54,10067,75481020,13.337128,7281,33549510,21.702254,35-54,2786,41931510,6.644168
2,2001,55-74,4056,33728474,12.025448,2921,10890782,26.820847,55+,1541,33261855,4.632935
3,2001,75+,1783,12449112,14.322307,1377,2024949,68.001713,.,.,.,.
4,2001,All,23580,186499780,12.643447,18004,78477150,22.941710,All,5576,108022630,5.161881


In [11]:
all_non_veterans = non_veterans[non_veterans['age_group'] == 'All']
all_non_veterans['deaths_growth'] = all_non_veterans['deaths'].pct_change()
all_non_veterans['rate_growth'] = all_non_veterans['rate_per_100k'].pct_change()
all_non_veterans['male_rate_growth'] = all_non_veterans['male_rate_per_100k'].pct_change()
all_non_veterans['female_rate_growth'] = all_non_veterans['female_rate_per_100k'].pct_change()
all_non_veterans

,year,age_group,deaths,population,rate_per_100k,male_deaths,male_population,male_rate_per_100k,female_age_group,female_deaths,female_population,female_rate_per_100k,deaths_growth,rate_growth,male_rate_growth,female_rate_growth
4,2001,All,23580,186499780,12.643447,18004,78477150,22.941710,All,5576,108022630,5.161881,NaN,NaN,NaN,NaN
9,2002,All,24529,189265736,12.960085,18630,80118723,23.252992,All,5899,109147013,5.404637,0.040246,0.025044,0.013568,0.047029
14,2003,All,24551,191968175,12.789099,18624,81663067,22.805903,All,5927,110305108,5.373278,0.000897,-0.013193,-0.019227,-0.005802
19,2004,All,25395,194707563,13.042637,18974,83253063,22.790753,All,6421,111454500,5.761095,0.034377,0.019825,-0.000664,0.072175
24,2005,All,25484,197376930,12.911337,19191,84763058,22.640759,All,6293,112613872,5.588122,0.003505,-0.010067,-0.006581,-0.030024
29,2006,All,26317,200459198,13.128357,19736,86545802,22.804110,All,6581,113913396,5.777196,0.032687,0.016809,0.007215,0.033835
34,2007,All,27505,203535802,13.513593,20552,88274976,23.281796,All,6953,115260826,6.032405,0.045142,0.029344,0.020947,0.044175
39,2008,All,28478,206625364,13.782432,21367,89949088,23.754549,All,7111,116676276,6.094641,0.035375,0.019894,0.020306,0.010317
44,2009,All,29320,209611362,13.987791,22015,91637196,24.024087,All,7305,117974166,6.192034,0.029567,0.014900,0.011347,0.015980
49,2010,All,30803,212437510,14.499793,23213,93186391,24.910290,All,7590,119251119,6.36472,0.050580,0.036603,0.036888,0.027889


In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=all_veterans['year'], y=all_veterans['rate_per_100k'], mode='lines', name='Veterans', line={'color': 'red'}))
fig.add_trace(go.Scatter(x=all_non_veterans['year'], y=all_non_veterans['rate_per_100k'], mode='lines', name='Non-veterans', line={'color': 'blue'}))
fig.update_layout(title='Suicide Rates for Veterans vs. Non-veterans',
                  yaxis_title='Rate per 100,000',
                  yaxis={'range': [0, 40]},
                  legend={'orientation': 'h', 'x': 0.3, 'y': -0.1})
fig.show()

In [13]:
t_stat, p_val = stats.ttest_ind(all_veterans['rate_per_100k'], all_non_veterans['rate_per_100k'])
print(f"T-statistic: {t_stat:.3f}")
print(f"P-value: {p_val:.5f}")

T-statistic: 16.952
P-value: 0.00000


## Question 3

Breaking down suicides and suicide rates by age group.

In [14]:
fig = px.line(data_frame=veterans[veterans['age_group'] != 'All'], x='year', y='deaths', color='age_group', title='Veteran Suicides by Age Group', range_y=(0, 3000))
fig.update_layout(yaxis_title='Suicides', xaxis_title='', legend={'orientation': 'h', 'x': 0.15, 'y': -0.1})
fig.show()

In [15]:
fig = px.line(data_frame=veterans[veterans['age_group'] != 'All'], x='year', y='rate_per_100k', color='age_group', title='Veteran Suicide Rates by Age Group', range_y=(0, 50))
fig.update_layout(yaxis_title='Suicides per 100k', xaxis_title='', legend={'orientation': 'h', 'x': 0.15, 'y': -0.1})
fig.show()

## Question 4

Determining whether there is a significant difference in suicide rates between male veterans and female veterans.

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=all_veterans['year'], y=all_veterans['male_rate_per_100k'], mode='lines', name='Males', line={'color': 'red'}))
fig.add_trace(go.Scatter(x=all_veterans['year'], y=all_veterans['female_rate_per_100k'], mode='lines', name='Females', line={'color': 'blue'}))
fig.update_layout(title='Veteran Suicide Rates for Males vs. Females',
                  yaxis_title='Suicides per 100,000',
                  yaxis={'range': [0, 40]},
                  legend={'orientation': 'h', 'x': 0.3, 'y': -0.1})
fig.show()

In [17]:
t_stat, p_val = stats.ttest_ind(all_veterans['male_rate_per_100k'], all_veterans['female_rate_per_100k'])
print(f"T-statistic: {t_stat:.3f}")
print(f"P-value: {p_val:.5f}")

T-statistic: 17.302
P-value: 0.00000


## Question 5

Breaking down suicide rates by demographic group.

In [18]:
by_demos.info()
demo_cols = ['year', 'white_deaths', 'white_rate', 'black_deaths', 'black_rate', 'native_deaths', 'native_rate', 'asian_deaths', 'asian_rate', 'multi_race_deaths', 'multi_race_rate', 'unknown_race_deaths', 'unknown_race_rate', 'hispanic_death', 'hispanic_rate']
by_demos.columns = demo_cols
by_demos = by_demos.drop(by_demos.index[:2])
by_demos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 15 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Year of Death                                 22 non-null     object
 1   White                                         23 non-null     object
 2   White2                                        23 non-null     object
 3   Black                                         23 non-null     object
 4   Black2                                        23 non-null     object
 5   American Indian / Alaskan Native              23 non-null     object
 6   American Indian / Alaskan Native2             23 non-null     object
 7   Asian, Native Hawaiian, or Pacific Islander   23 non-null     object
 8   Asian, Native Hawaiian, or Pacific Islander2  23 non-null     object
 9   Multiple Race                                 23 non-null     object
 10  Mult

,year,white_deaths,white_rate,black_deaths,black_rate,native_deaths,native_rate,asian_deaths,asian_rate,multi_race_deaths,multi_race_rate,unknown_race_deaths,unknown_race_rate,hispanic_death,hispanic_rate
2,2001,5154,23.3,292,11.7,44,25.9,34,10.8,<10,--,468,7.8,149,13.3
3,2002,5318,24.5,281,11,31,18.9,28,8.8,13,3.6,471,7.7,151,13.2
4,2003,5210,24.4,257,10.1,34,17.8,39,12,<10,--,464,7.7,149,12.4
5,2004,5148,24.4,291,11.3,30,16.5,38,11.6,16,5.5,481,8,162,13.6
6,2005,5293,25.5,287,11,38,20.9,41,11.8,<10,--,460,7.5,152,12.7
7,2006,5266,25.9,261,9.9,32,17.3,50,14.7,<10,--,419,6.9,145,12
8,2007,5415,27.2,283,10.9,33,18.5,47,14.3,<10,--,464,7.4,170,14.2
9,2008,5462,27.7,284,11,32,19,51,15.7,14,4.1,724,11,153,12.2
10,2009,5212,26.9,286,11.2,45,26.9,68,21.6,12,3.4,896,13.7,164,13
11,2010,5875,30.8,342,13,37,22.3,81,25.6,17,4.7,193,2.9,205,16


In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=by_demos['year'], y=by_demos['white_rate'], name='White'))
fig.add_trace(go.Scatter(x=by_demos['year'], y=by_demos['black_rate'], name='Black'))
fig.add_trace(go.Scatter(x=by_demos['year'], y=by_demos['hispanic_rate'], name='Hispanic'))
fig.add_trace(go.Scatter(x=by_demos['year'], y=by_demos['asian_rate'], name='Asian'))
fig.add_trace(go.Scatter(x=by_demos['year'], y=by_demos['native_rate'], name='Native American'))
fig.update_layout(title='Veteran Suicide Rates by Demographic',
                  yaxis_title='Suicides per 100,000',
                  yaxis={'range': [0, 50]},
                  legend={'orientation': 'h', 'x': 0.05, 'y': -0.15})
fig.show()

# Conclusion

1. The first question explored the overall number of suicides and suicide rates among veterans from 2001-2021. The minimum number of suicides was 6,000 in 2001. The maximum was 6,718 in 2018. The absolute number of suicide deaths grew at an average annual rate of 0.34% from 2001-2021. The minimum suicide rate (per 100k) was 23.26 in 2001, and the maximum was 33.93 in 2021. The growth rate in suicide rates was 1.93%. The suicide rate growth seemed rather high to me so I conducted an OLS regression on suicide rates vs. year. There was indeed a statistically significant rate of increase in suicide rates. The intercept coefficient and the year coefficient were both significant at the 99% level of confidence. So that's an unsettling finding because it shows that veteran suicides are growing at a high rate. I also used the regression results to create a 5 year forecast of suicide rates, and as expected, they are predicted to continue increasing. 

2. The next question compared the suicide rates between veterans and non-veterans. This was another troubling statistic since as we can see in the visualization above, veteran suicide rates are over double the non-veteran suicide rates throughout the entire 20 year period. I also conducted a t-test to determine if there was a statistically significant difference between the 2 populations, which just reinforced what we could already discern. There was a significant difference in suicide rates between the 2 populations at a 99% confidence level. Pretty sad statistic for the veterans. 

3. The third question looked at suicide numbers and suicide rates broken down by age group. For overall numbers, the 18-34 age group is lowest throughout the time series (which surprised me). The 55-74 age group overtook 35-54 in 2007 and has been the highest ever since. And the 35-54 age group slowly declined in numbers throughout the time series, and is the only age group to have done so. The suicide rates however, tell a different story. The 18-34 age group became the highest in 2010 and has continually separated from the others with a much higher rate of increase. All other groups show a smaller rate of increase throughout, but they all increase none-the-less unfortunately.

4. The fourth question compared suicide rates between male veterans and female veterans. These results are quite similar to those between veterans and non-veterans. The male veteran rates are over double the female veteran rates throughout the entire time series. And as expected, a t-test showed a statistically significant difference between the groups at a 99% confidence level. 

5. The fifth question looked at suicide rates broken down by demographic group. While all groups showed increase throughout the time series, blacks displayed the smallest rate, followed by hispanics. The next three groups (asian, native american, white) showed a large jump up from the other two. Whites had the highest rates throughout all but two of the years. Native americans were second to whites, except for 2014 and 2021, where native americans jumped above the entire set. 

Final thoughts: These results were quite upsetting. According to this report, the average veteran is more than doubly at risk for suicide than the average non-veteran. The average male veteran is more than doubly at risk than the average female veteran. The average 18-34 year old veteran is more at risk than an average veteran of any other age. And the average white veteran and native american veteran is more at risk than the other demographic groups. So, a white or native american, 18-34 year old male veteran is a very vulnerable group for suicide risk. 

## Resources

https://www.mentalhealth.va.gov/docs/data-sheets/2021/VA_National_2001-2021_Appendix_508.xlsx

https://department.va.gov/suicide-prevention-annual-report/

https://plotly.com/python/